We first install and import necessary packages.

In [1]:
!pip install torch
!pip install gym==0.21
!pip install stable-baselines3==1.6.2

     |████████████████████████████████| 137.6 MB 267 kB/s            
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
You should consider upgrading via the '/Users/giovanniaiello/Dropbox/00/RL/Assignments/Assignment 3/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.5 MB 3.7 MB/s            
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.23.4-cp38-cp38-macosx_10_9_x86_64.whl (18.1 MB)
  Using cached cloudpickle-2.2.0-py3-none-any.whl (25 kB)
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=23eac5ae7fdaa3f13595f467f2a8c930a0c4bc9f6d2910fbc87416ed77307421
  Stored in directory: /Users/giovanniaiello/Library/Caches/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
You should consider upgrading via the '/Users/giovanniaiello/Dropbox/00/RL/Assignments/Assignment 3/venv/bin/python -m pip install --upgrade pip' command.
     |███████████

In [7]:
import gym

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as functional

import matplotlib.pyplot as plt

## Acrobot

Next, we instantiate the [Acrobot environment](https://www.gymlibrary.dev/environments/classic_control/acrobot/) from OpenAI Gym and gain a quick understanding of its key variables and methods. 

The Acrobot environment includes a simple robot with two blue links that are connected by two green joints. The joint connecting the two links is actuated, i.e., it can be controlled by the robot by applying torque to the joint. The goal is to swing the free end of the outer link to reach the target height (shown as the black horizontal line) by using robot's actuation.

Follow the hyperlinks to learn more about the envionment [Acrobot environment](https://www.gymlibrary.dev/environments/classic_control/acrobot/)  and its [source code](https://github.com/openai/gym/blob/master/gym/envs/classic_control/acrobot.py).

![](https://www.gymlibrary.dev/_images/acrobot.gif)

In [3]:
env = gym.make("Acrobot-v1")

#### States

OpenAI Gym environments do not explicitly provide state but instead provide observation. In the Acrobot environment, the state and observation are identical. The state of the Acrobot is a 6-dimensional vector, which provides information about the two rotational joint angles of the robot and their angular velocities. As joint angles and velocity are continuous variables, so is the Acrobot state.

In [4]:
print(f"Number of state features: {env.observation_space.shape[0]}")

Number of state features: 6


#### Actions

In contrast to the continuous state, the action space for this robot is discrete. The robot has only three choices:
- apply torque of unit 1 in clockwise direction,
- apply torque of until 1 in counter-clockwise direction, or
- apply no torque.

In [5]:
print(f"Number of actions: {env.action_space.n}")

Number of actions: 3


#### Transition and Reward Function

The transition and rewards functions for the environment are not explicitly represented as matrices or tensors. Instead the gym API provides access to the `step` method, which takes in as input an `action` and provides (among other things) an `observation` and `reward`. Also notice the `reset` method, which resets the MDP environment.

The following snippet describes the use of `step` and `reset` methods.

In [6]:
# Reset the environment
state = env.reset() 
print(f"State: {state}")

# Select a random action to play
action = env.action_space.sample()
print(f"Action: {action}")

# Send this action to the environment to receive the next state and reward
next_state, reward, done, _ = env.step(action)
print(f"Next state: {next_state}")
print(f"Reward: {reward}")

State: [ 0.9957404   0.092201    0.9999109   0.0133515  -0.01067897 -0.07086682]
Action: 1
Next state: [ 0.9968618   0.07916107  0.99997747  0.00671444 -0.11764181  0.00343792]
Reward: -1.0


## Problem 1.1

Next, we will implement the general recipe of Q Learning algorithm to compute the optimal policy in the Acrobot environment. 

#### Q Network

Since, the robot's state space is continuous, we cannot represent the Q value function exactly. Instead, we will approximate the Q value as a neural network. Let us first define this neural network. 

Many choices exist for the neural network architecture. We will utilize a multi-layer perceptron, with
- input as the state of the Acrobot, and
- output as a vector of size 3 denoting Q values of each action for the input state.

In [ ]:
class QNetwork(nn.Module):
    """Approximates the Q Function as a Multi-Layer Perceptron."""

    def __init__(self, env, nodes_per_mlp_layer=[128, 64]):
        """Initialize the Q Function apprixmated as a Multi-Layer Perceptron.
        
        Args:
            env: An OpenAI Gym environment.
            nodes_per_mlp_layer: An array of integers. The length of array equals the number
                of hidden layers of the Multi-Layer Perceptron. Each element in the array
                equals the number of nodes in the corresponding layer.
        """
        super().__init__()
        
        ######## PUT YOUR CODE HERE ########
        # input layer
        self.input_layer = nn.Linear(NotImplemented, NotImplemented)
        
        # hidden layers
        self.hidden_layers = nn.ModuleList()   
        for k in range(len(nodes_per_mlp_layer) - 1):
            self.hidden_layers.append(nn.Linear(NotImplemented, NotImplemented))

        # output layer
        self.output_layer = nn.Linear(NotImplemented, NotImplemented)
        ######## PUT YOUR CODE HERE ########

    def forward(self, x):
        """Implements the forward pass of the Q Network.
        
        Args:
            x: Input to the Q Network.
        
        Returns:
            Output of the Q Network.
        """
        ######## PUT YOUR CODE HERE ########
        
        ######## PUT YOUR CODE HERE ########

#### Replay Buffer

In addition to the Q Network, our algorithm requires a data structure to store the agent's experiences: Replay Buffer. Let us now define this data structure.

Similar to Q Network, many data structure choices exist for the Replay Buffer. We will use a simple buffer to store agent's experiences.

In [ ]:
class ReplayBuffer:
    """A buffer to store agent's experiences."""
    
    def __init__(self, env, buffer_size):
        """Initialize a ring buffer to store agent's experiences.
        
        Args:
            env: An OpenAI Gym environment.
            buffer_size: An integer. The total size of the buffer.
        """
        observation_n = env.observation_space.shape[0]
        self.buffer_size = buffer_size
        
        self.observations = np.zeros((self.buffer_size, observation_n), dtype=np.float32)
        self.next_observations = np.zeros((self.buffer_size, observation_n), dtype=np.float32)
        self.actions = np.zeros((self.buffer_size,), dtype=np.int64)
        self.rewards = np.zeros((self.buffer_size,), dtype=np.float32)
        self.dones = np.zeros((self.buffer_size,), dtype=np.float32)

        ######## PUT YOUR CODE HERE ########

        ######## PUT YOUR CODE HERE ########
    
    def add(self, state, action, next_state, reward, done):
        """Add an experience to the buffer.
        
        Args:
            state: the current environment state
            action: the action executed in the state
            next_state: the state after executing the action
            reward: the reward received after executing the action
            done: Boolean denoting whether the task is completed.        
        """
        ######## PUT YOUR CODE HERE ########

        ######## PUT YOUR CODE HERE ########

    def sample(self, batch_size):
        """Sample a mini-batch of experiences from the replay buffer.
        
        Args:
            batch_size: An integer. The size of the mini-batch.
        
        Returns:
            Randomly sampled experiences from the replay buffer.
        """
        indices = np.random.randint(self.size, size=batch_size)
        observations = torch.from_numpy(self.observations[indices])
        next_observations = torch.from_numpy(self.next_observations[indices])
        actions = torch.from_numpy(self.actions[indices])
        rewards = torch.from_numpy(self.rewards[indices])
        dones = torch.from_numpy(self.dones[indices])

        return observations, actions, next_observations, rewards, dones

#### Exploration schedule

Q Learning typically utilizes epsilon-greedy strategy for exploration. 

In general, it is useful to explore more initially (when the agent is far from the optimal policy) and less later on in the learning process. To implement this, we can implement a schedule for epsilon.

Similar to the Q network and replay buffer architectures, several choices exist for determining the epsilon schedule. We will use a simple linear scheduler to decay epsilon as agent gains more experience. 

In [ ]:
def linear_schedule_for_epsilon(initial_value, final_value, duration, timestep):
    """Implements a linear scheduler for epsilon.
    
    Args:
        initial_value: (float) Initial value of epsilon.
        final_value: (float) Final value of epsilon.
        duration: (int) Duration over which to decay epsilon from its initial to final value.
        timestep: (int) The current time step.
    
    Returns:
        Value of epsilon at the given timestep.
    """
    slope = (final_value - initial_value) / duration
    return max(slope * timestep + initial_value, final_value)

#### General recipe for Q Learning with Function Approximation

Now that we have all the building blocks, we will implement the general recipe for Q learning with function approximation. 

We will use the AgentBase class from Assignment 2 to help us with the implementation.

In [ ]:
class AgentBase:
    def __init__(self, env):
        self.env = env
        self.num_actions = self.env.action_space.n
        self.policy = self.make_policy()
        self.behavior_policy = self.make_behavior_policy()

    def make_policy(self):
        """
        Return a policy function that will be used for evaluation. The policy
        takes observation as input and return action
        """
        raise NotImplementedError

    def make_behavior_policy(self):
        """
        Similar to make_policy, it returns a policy function. But this one used
        for interaction with the environment.
        """
        raise NotImplementedError

    def run_episode(self, episode_policy):
        """
        Generate one episode with the given policy
        """
        episode = []
        done = False
        obs = self.env.reset()
        episode_return = 0
        while not done:
            action = episode_policy(obs)
            next_obs, reward, done, _ = self.env.step(action)
            episode.append([obs, action, reward, next_obs, done])
            obs = next_obs
            episode_return += reward

        return (episode, episode_return)

    def evaluate(self, num_eval_episodes=1000, plot_title="Evaluation"):
        """Evaluates the agent."""
        list_returns = []
        list_average_returns = []
        average_return = 0
        for episode_idx in range(num_eval_episodes):
            _, episode_return = self.run_episode(self.policy)
            average_return += (1. / (episode_idx+1)) * (episode_return - average_return)
            list_returns.append(episode_return)
            list_average_returns.append(average_return)

        print(f"Average reward {round(average_return, 3)}")
        plt.plot(list_returns,'^',label="Return")
        plt.plot(list_average_returns,'r',label="Average Return")
        plt.ylabel('Return')
        plt.xlabel('Episode#')
        plt.title(plot_title)
        plt.legend()
        plt.ylim(-501, 0.0)
        plt.show()

In [ ]:
class DeepQLearning(AgentBase):
    """Implements a Q Learner with function approximation."""
    
    def __init__(self,
        env,
        buffer_size = 200000,
        batch_size = 64,
        initial_epsilon = 1.0,
        final_epsilon = 0.01,
        epsilon_decay_duration = 100000,
        learning_rate = 0.001,
        num_gradient_updates = 1,
        q_network_update_frequency = 1,
        target_network_update_frequency = 200,
        learning_starts_at_step = 10000):
        """Initializes the Agent.
        
        Args:
            env: An OpenAI Gym environment.
            buffer_size: (integer) Size of the replay buffer.
            batch_size: (integer) Size of the mini batch.
            initial_epsilon: (float) Initial value of epsilon.
            final_epsilon: (float) Final value of epsilon.
            epsilon_decay_duration: (integer) Duration over which to decay epsilon.
            learning_rate: (float) Learning rate for Q network update.
            num_gradient_updates: (integer) Number of stochastic gradient updates with each minibatch.
            q_network_update_frequency: (integer) Steps after which to update Q network.
            target_network_update_frequency: (integer) Steps after which to update target network.            
            learning_starts_at_step: (integer) Step at which to begin learning. Before this, the 
                agent explores and collects experiences in its replay buffer.
        """
        super().__init__(env=env)
        
        self.gamma = 0.99 # Assume a discount factor of 0.99
        self.current_step = 0
        self.learning_starts_at_step = learning_starts_at_step
        self.batch_size = batch_size
        self.num_gradient_updates = num_gradient_updates
        self.q_network_update_frequency = q_network_update_frequency
        self.target_network_update_frequency = target_network_update_frequency

        # Create exploration scheduler
        self.epsilon_scheduler = lambda current_step: linear_schedule_for_epsilon(
            initial_epsilon, final_epsilon, epsilon_decay_duration, current_step)      

        ######## PUT YOUR CODE HERE ########
        
        ######## PUT YOUR CODE HERE ########

        self.state = env.reset()
        self.list_returns = []
        self.list_average_returns = []
        self.average_return = 0.
        self.list_moving_average_returns = []
        self.moving_average_returns_by_step = np.empty([200000])
    
    @property
    def epsilon(self):
        return self.epsilon_scheduler(self.current_step)

    def make_policy(self):
        def policy_func(observation):
            ######## PUT YOUR CODE HERE ########
            
            ######## PUT YOUR CODE HERE ########

        return policy_func

    def make_behavior_policy(self):
        def policy_func(observation):
            ######## PUT YOUR CODE HERE ########
            
            ######## PUT YOUR CODE HERE ########

        return policy_func
    
    def update(self):
        """Update the agent."""                      
        ######## PUT YOUR CODE HERE ########
        
        ######## PUT YOUR CODE HERE ########
        
    
    def train(self, num_train_episodes, make_plot=False):
        
        for episode_idx in range(num_train_episodes):
            # Reset environment before beginning the episode
            done = False
            self.state = self.env.reset()
            episode_return = 0
            
            # Run the episode and update the policy 
            while not done:
                # First, generate a step with behavior policy
                action = self.behavior_policy(self.state)
                next_state, reward, done, _ = self.env.step(action)
                
                # Update the replay buffer
                self.replay_buffer.add(self.state, action, next_state, reward, done)
            
                # Second, update the agent
                self.update()
                
                # Prepare for next step
                self.state = next_state
                self.current_step += 1
                episode_return += reward
                if self.current_step < 200000:
                    if len(self.list_moving_average_returns) > 0:
                      self.moving_average_returns_by_step[self.current_step] = self.list_moving_average_returns[-1]
                    else:
                      self.moving_average_returns_by_step[self.current_step] = -500.

                if self.current_step % 10000 == 0:
                    print(f"Timestep: {self.current_step}, episode reward (moving average, 20 episodes): {round(self.list_moving_average_returns[-1],2)}")
            
            # Store the return for evaluation
            self.list_returns.append(episode_return)
            self.average_return = np.mean(np.asarray(self.list_returns))
            self.list_average_returns.append(self.average_return)

            if len(self.list_returns) > 20:
                self.list_moving_average_returns.append(
                  np.mean(np.asarray(self.list_returns[-20:])))
            else:
                self.list_moving_average_returns.append(self.average_return)

        if make_plot:
            plt.plot(self.list_returns,'^',label="Return")
            plt.plot(self.list_average_returns,'r',label="Average Return (all episodes)")
            plt.plot(self.list_moving_average_returns,'b',label="Average Return (last 20 episodes)")
            plt.ylabel('Return')
            plt.xlabel('Episode#')
            plt.title('Performance during training')
            plt.ylim(-501, 0.0)
            plt.legend()        
            plt.show()        

#### Training the Agent

Having implemented the agent, now we will train it using the default hyperparameters provided in the class definition and observe its performance.

In [ ]:
env = gym.make("Acrobot-v1")
agent_001 = DeepQLearning(env)

In [ ]:
agent_001.evaluate(plot_title="Performance before training", num_eval_episodes=100)

In [ ]:
agent_001.train(1000, make_plot=True)

In [ ]:
agent_001.evaluate(plot_title="Performance after training", num_eval_episodes=100)

## Problem 1.2

Now that we have implemented the general recipe, let us run ablation studies to assess importance of different components of the Deep Q Learning algorithm.

First, we will implement and assess the performance of the original Q Learning with naive function approximation, i.e., without the use of replay buffer and target networks. Note that this is a special case of our general recipe and, in general, we do not expect to work in practice. Let us see if it works for the Acrobot environment.

In [ ]:
class OnlineDeepQLearning(DeepQLearning):
    """Implements Online Q Learner with function approximation."""
    
    ######## PUT YOUR CODE HERE ########

    ######## PUT YOUR CODE HERE ########

In [ ]:
env = gym.make("Acrobot-v1")
agent_002 = OnlineDeepQLearning(env)

In [ ]:
agent_002.train(500, make_plot=True)

Now let us bring back one of the components at a time that made the algorithm successful in Problem 1. First, we will bring back only the replay buffer (and not the target network) and observe its effect on the agent's performance.

In [ ]:
class DeepQLearningWithoutTargetNetwork(DeepQLearning):
    """Implements a Deep Q Learner without target network."""
    
    ######## PUT YOUR CODE HERE ########

    ######## PUT YOUR CODE HERE ########

In [ ]:
env = gym.make("Acrobot-v1")
agent_003 = DeepQLearningWithoutTargetNetwork(env)

In [ ]:
agent_003.train(1000, make_plot=True)

Next, we will bring back only the target network (and not the replay buffer) and observe its effect on the agent's performance.

In [ ]:
class DeepQLearningWithoutReplayBuffer(DeepQLearning):
    """Implements a Deep Q Learner without replay buffer."""
    ######## PUT YOUR CODE HERE ########

    ######## PUT YOUR CODE HERE ########

In [ ]:
env = gym.make("Acrobot-v1")
agent_004 = DeepQLearningWithoutReplayBuffer(env)

In [ ]:
agent_004.train(1000, make_plot=True)

In a single plot, show the performance of the four variants that you have implemented above. Specifications of the plot
- X axis: Number of Timesteps (0 to 100000)
- Y axis: 20-episode simple moving average of episodic rewards.

In [ ]:
######## PUT YOUR CODE HERE ########

######## PUT YOUR CODE HERE ########

## Problem 1.3

Select any one hyperparameter of the DeepQLearning agent and study its effect on the agent's performance. You should try at least five different values of this hyperparameter. In a single plot, show the performance of the agent for different values of hyperparameter.  Specifications of the plot
- X axis: Number of Timesteps (0 to 100000)
- Y axis: 20-episode simple moving average of episodic rewards.


In [ ]:
######## PUT YOUR CODE HERE ########

######## PUT YOUR CODE HERE ########

## Problem 1.4

While applying RL in real world, you may consider the use of off-the-shelf implementation of RL algorithms. The package [Stable Baselines 3](stable-baselines3.readthedocs.io/) aims to provide a set of reliable implementations of RL algorithms in PyTorch. 

Familiarize yourself with this package and use its implementation of Deep Q Network to learn the optimal policy for the Acrobot environment.

In [ ]:
######## PUT YOUR CODE HERE ########

######## PUT YOUR CODE HERE ########

In a single plot, compare the performance of your implementation of the algorithm in Problem 1.1. with the one that you implemented above (Problem 1.4). Specifications of the plot
- X axis: Number of Timesteps (0 to 100000)
- Y axis: 20-episode simple moving average of episodic rewards.

In [ ]:
######## PUT YOUR CODE HERE ########

######## PUT YOUR CODE HERE ########